# OpenML Impact Analysis

In [ ]:
!pip install pandas --quiet
!pip install matplotlib --quiet

In [3]:
import pandas as pd
import matplotlib.pyplot as plt


## Data cleaning and preprocessing

In [ ]:
path1 = '../data/collected_papers.csv' # Original list of collected papers
path2 = "../data/Final_survey_data.csv" # Survey Results

df = pd.read_csv(path2)

print("Total no of reviews recieved: ", len(df))
df.head()

In [ ]:
pd.DataFrame(df.columns)

### Sanity Check:
1. Remove duplicates
2. Match paper ID and paper title from original list
3. Remove papers with wrong year (<2014)
4. Remove empty rows

In [ ]:
Total_papers = pd.read_csv(path1) # Original list of scraped paper
print("Total scraped papers: ", len(Total_papers))

In [ ]:
assigned_papers_dict = Total_papers[["Paper ID", "Title"]].set_index('Paper ID').to_dict()
assigned_papers_dict = assigned_papers_dict["Title"]
assigned_papers_dict

In [ ]:
sheet1 = Total_papers[["Paper ID", "openml-suites-2021", "openml-python-2021", "openml-2014", "openml-r-2017"]]
sheet1.rename(columns={"Paper ID": "Paper ID (from shared sheet)"}, inplace=True)
df = df.merge(sheet1, on="Paper ID (from shared sheet)", how="left")
df.head()

In [ ]:
# Step 1: Remove Dublicates

# Duplicate entries
duplicate_rows = df[df.duplicated(subset=["Paper ID (from shared sheet)"], keep=False)]

print(len(duplicate_rows))
duplicate_rows

In [ ]:
duplicates = df[df.duplicated(subset=["Paper ID (from shared sheet)"], keep=False)]
# Save duplicates to a separate DataFrame before removal
duplicates_removed = df[df.duplicated(subset=["Paper ID (from shared sheet)"], keep="first")]

df = df.drop_duplicates(subset=["Paper ID (from shared sheet)"], keep="first")

print(len(duplicates_removed))
duplicates_removed
# ToDo - better strategy to remove duplicates - ask reviewers to correct them.

In [ ]:
print("No of reviews after dublicate removal: ", len(df))

In [ ]:
# Step 2: Remove mismatches

mismatches = []

for index, row in df.iterrows():
    try:
        paper_id = row["Paper ID (from shared sheet)"]
        paper_title = row["Paper Title"].strip().lower()
        assigned_title = assigned_papers_dict.get(paper_id).strip().lower()

        if paper_title not in assigned_title:
            mismatches.append({
                "Paper ID": paper_id,
                "Given Title": row["Paper Title"],
                "Expected Title": assigned_papers_dict.get(paper_id, "Not Found")
            })

    except Exception as e:
        mismatches.append({
            "Paper ID": row["Paper ID (from shared sheet)"],
            "Error": str(e),
            "Given Title": row.get("Paper Title", "Not Found")
        })
        continue

mismatches_df = pd.DataFrame(mismatches)
mismatches_df



In [ ]:
# Mismatched paper ID and title - 901, 240, 881, 1467, 1472, 108 - to be removed
paper_ids_to_remove = [901, 240, 881, 1467, 1472, 108] #contact authors

df = df[~df["Paper ID (from shared sheet)"].isin(paper_ids_to_remove)]
df[df["Paper ID (from shared sheet)"] == 108]

In [ ]:
print("No of reviews after duplicate removal: ", len(df))

In [ ]:
# Drop rows with all NaN or NaT values
df = df.dropna(how="all")

len(df)

In [ ]:
# Remove papers with a year before 2014 - temporary messure. some incorrect paper year.

# df["Paper Year"] = df["Paper Year"].astype(int)

# # Check the updated column type
# print(df["Paper Year"].dtypes)

df = df[df["Paper Year"] >= 2014]
len(df)

In [ ]:
print("Paper with wrong year: ", 1696-1665)

In [ ]:
count_yes = df["Paper Available"].value_counts().get("Yes", 0)
print(count_yes)
print(len(df)-count_yes + (len(Total_papers)-1702))

In [ ]:
count_yes = df["Paper in English"].value_counts().get("Yes", 0)
print(count_yes)
print(len(df)-count_yes)


In [ ]:
# No fo paper which are both not fully available and not in english 
overlap = df[(df["Paper Available"] != "Yes") & (df["Paper in English"] != "Yes")]
len(overlap)

In [ ]:

columns_to_analyze = ["openml-suites-2021", "openml-python-2021", "openml-2014", "openml-r-2017"]

# Calculate the percentage of True values for each column
percentages = Total_papers[columns_to_analyze].mean() * 100

# Print the results
for column, percentage in percentages.items():
    print(f"Percentage of papers with True for {column}: {percentage:.2f}%")


In [ ]:
# Filter rows not in final_df
not_in_final_df = df[~((df["Paper Available"] == "Yes") &  (df["Paper in English"] == "Yes"))]

print(len(not_in_final_df))
# Display the rows not in final_df
not_in_final_df[["Paper Available", "Paper in English"]]


In [ ]:
# Should only consider paper available + in english
available_papers = df[(df["Paper Available"] == "Yes") & (df["Paper in English"] == "Yes")]
print("Final number of papers for analysis: ", len(available_papers))

Final statistics

1. Total exctracted paper: 1719
2. Paper used in analysis: 1482

## Analysis

In [ ]:

# Count the number of papers per year
papers_per_year = available_papers["Paper Year"].value_counts().sort_index()

plt.figure(figsize=(10, 5))
plt.bar(papers_per_year.index, papers_per_year.values, color="skyblue", edgecolor="black")

plt.xlabel("Year")
plt.ylabel("Number of Papers")
plt.title("Number of OpenML Cited Papers Per Year")
plt.xticks(rotation=45)
plt.grid(linestyle='--', alpha=0.2)

plt.show()


In [ ]:
# What percentage of the papers does just cite OpenML, but not actively interact with it?

interaction_columns = [
    "Does the paper use datasets from OpenML?",
    "Does the paper use a collection (at least 2 or more) of datasets that are defined by OpenML designated to do benchmarking (e.g., openml benchmarking suites)?",
    "Does the paper use OpenML experiment data (i.e., utilise results from runs)? ",
    "Does the paper upload datasets to OpenML?",
    "Does the paper upload experiment data to OpenML?",
    "Does the paper interact in any other way with OpenML?"
]

just_citing_papers = available_papers[(available_papers[interaction_columns] == "No").all(axis=1)]

percentage_citing_only = (len(just_citing_papers) / len(available_papers)) * 100
print(len(just_citing_papers), round(percentage_citing_only, 2))

core_citing_papers = just_citing_papers[just_citing_papers["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]
print(len(core_citing_papers), round((len(core_citing_papers)/len(just_citing_papers))*100,2))


In [ ]:
# out of above papers which just cites openml, % of paper citing Openml-2014 paper

just_citing_openml_2014 = just_citing_papers[just_citing_papers["openml-2014"] == True]
print(len(just_citing_openml_2014), (len(just_citing_openml_2014) / len(just_citing_papers)) * 100)


In [ ]:
english_counts = df["Paper in English"].value_counts()

plt.figure(figsize=(18, 4))

plt.subplot(1, 2, 1)
plt.pie(english_counts, labels=english_counts.index, autopct="%1.1f%%")
plt.title("Papers in English")


core_member_counts = available_papers["Does the paper have at least 1 current OpenML Core Member as co-author?"].value_counts()

plt.subplot(1, 2, 2)
plt.pie(core_member_counts, labels=core_member_counts.index, autopct="%1.1f%%")
plt.title("Papers with OpenML Core Member Co-author")

plt.tight_layout()
plt.show()

In [ ]:
# Datasets

dataset_papers = available_papers[available_papers["Does the paper use datasets from OpenML?"] == "Yes"]
dataset_percentage = (len(dataset_papers) / len(available_papers)) * 100
print(dataset_percentage, (len(dataset_papers)))
print("Number of paper by core-authors: ",len(dataset_papers[dataset_papers["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]))

unclear_dataset_papers = available_papers.dropna(subset=["If unclear, one sentence explanation?"])
unclear_dataset_papers = unclear_dataset_papers[["Does the paper use datasets from OpenML?", "If unclear, one sentence explanation?"]]
unclear_dataset_papers

In [ ]:
# Benchmark
benchmark_papers = available_papers[available_papers["Does the paper use a collection (at least 2 or more) of datasets that are defined by OpenML designated to do benchmarking (e.g., openml benchmarking suites)?"] == "Yes"]
benchmark_percentage = (len(benchmark_papers) / len(available_papers[available_papers["Paper Year"] >= 2017])) * 100
print(benchmark_percentage, len(benchmark_papers))

benchmark_datasets = benchmark_papers["If yes, which benchmark suites?"]

print("Number of paper by core-authors: ",len(benchmark_papers[benchmark_papers["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]))
# Print list of benchmark datasets
unique_benchmark_datasets = (
    benchmark_datasets.str.lower().str.strip().unique()
)


unique_benchmark_datasets

In [ ]:
#  Experiment data (Runs)

experiment_papers = available_papers[available_papers["Does the paper use OpenML experiment data (i.e., utilise results from runs)? "] == "Yes"]
experiment_percentage = (len(experiment_papers) / len(available_papers)) * 100

print(len(experiment_papers) ,experiment_percentage)
print("Number of paper by core-authors: ",len(experiment_papers[experiment_papers["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]))


experiment_papers[["Paper ID (from shared sheet)", "if yes: short (e.g., 1 sentence) explanation: how does it use this data?"]]


In [ ]:
upload_datasets = available_papers[available_papers["Does the paper upload datasets to OpenML?"] == "Yes"]

upload_datasets_percentage = len(upload_datasets) / len(available_papers) * 100

print( len(upload_datasets), upload_datasets_percentage)
print("Number of paper by core-authors: ",len(upload_datasets[upload_datasets["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]))


upload_datasets[["Paper ID (from shared sheet)", "Does the paper upload datasets to OpenML?", "If yes, which dataset?"]]

In [ ]:
upload_experiment_data_papers = available_papers[available_papers["Does the paper upload experiment data to OpenML?"] == "Yes"]

upload_experiment_data_percentage = len(upload_experiment_data_papers) / len(available_papers) * 100

print(len(upload_experiment_data_papers),upload_experiment_data_percentage)
print("Number of paper by core-authors: ",len(upload_experiment_data_papers[upload_experiment_data_papers["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]))

upload_experiment_data_papers[["Paper ID (from shared sheet)", "if yes: short (e.g., 1 sentence) explanation: what type of experiments?"]]

In [ ]:
# paper year not mentioned.
available_papers[available_papers["Paper Year"].isna()]

In [ ]:

available_papers["Paper Year"] = available_papers["Paper Year"].astype(int)

available_papers["Datasets Used"] = available_papers["Does the paper use datasets from OpenML?"] == "Yes"
available_papers["Benchmark Used"] = available_papers["Does the paper use a collection (at least 2 or more) of datasets that are defined by OpenML designated to do benchmarking (e.g., openml benchmarking suites)?"] == "Yes"

# Group data by year for visualization
datasets_by_year = available_papers.groupby("Paper Year")["Datasets Used"].sum()
benchmarks_by_year = available_papers.groupby("Paper Year")["Benchmark Used"].sum()

# Filter out the year 2025 from the data for plotting
datasets_by_year_filtered = datasets_by_year[datasets_by_year.index != 2025]
benchmarks_by_year_filtered = benchmarks_by_year[benchmarks_by_year.index != 2025]

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(
    datasets_by_year_filtered.index,
    datasets_by_year_filtered.values,
    color="blue",
    marker="o",
    label="Papers Using Openml Datasets'"
)

# Add data labels for datasets
for i, value in enumerate(datasets_by_year_filtered.values):
    ax.text(datasets_by_year_filtered.index[i], value, str(value), color="blue", fontsize=10, ha="right", va="bottom")

# Line plot for benchmarks used
ax.plot(
    benchmarks_by_year_filtered.index,
    benchmarks_by_year_filtered.values,
    color="red",
    marker="o",
    label="Papers Using Openml Benchmarking Suites"
)

# Add data labels for benchmarks
for i, value in enumerate(benchmarks_by_year_filtered.values):
    ax.text(benchmarks_by_year_filtered.index[i], value, str(value), color="red", fontsize=10, ha="right", va="bottom")


# # Add horizontal annotations on the x-axis
# annotations = {
#     2016: "openml R",
#     2017: "preprint of OpenML Benchmarking Suites paper",
#     2018: "openml-python on PyPI",
#     2019: "first paper for AutoML benchmark, openml R paper",
#     2021: "NeurIPS paper on OpenML Benchmarking Suites, OpenML Python paper"
# }

# import textwrap

# # Add vertical lines and annotations
# for year, annotation in annotations.items():
#     # Split the annotation into multiple lines if it's too long
#     wrapped_text = textwrap.fill(annotation, width=30)  # Adjust width based on your preference

#     # Add the vertical line
#     ax.vlines(x=year, ymin=0, ymax=max(100, 100) * 1.05, color='green', linestyle='--', lw=1)

#     # Place the annotation text horizontally at the top of the vertical line
#     ax.text(year, max(150, 150) * 1.05, wrapped_text, ha='center', va='top', fontsize=10, color='green', rotation=90)


ax.set_xticks(range(min(datasets_by_year.index), max(datasets_by_year.index)))
ax.set_xticklabels(range(min(datasets_by_year.index), max(datasets_by_year.index)), rotation=45)

# Labels and title
ax.set_xlabel("Year")
ax.set_ylabel("Number of Papers")
ax.set_title("Datasets and Benchmarks Used by Papers per Year")
ax.legend()

ax.grid(True, linestyle="--", alpha=0.6)

plt.tight_layout()
plt.show()



In [ ]:
other_interactions = available_papers[available_papers["Does the paper interact in any other way with OpenML?"] == "Yes"]

other_interactions_percentage = len(other_interactions) / len(available_papers) * 100

print(len(other_interactions) , other_interactions_percentage)
print("Number of paper by core-authors: ",len(other_interactions[other_interactions["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]))


other_interactions[["Does the paper interact in any other way with OpenML?", "if yes: short (e.g., 1 sentence) explanation how?"]]

In [ ]:
starred_papers = available_papers[available_papers["Star it as some cool project to be showcased in our paper?"] == "Yes"]

print(len(starred_papers))

starred_papers[["Paper ID (from shared sheet)", "If yes, please motivate your answer"]]


In [108]:
# output_file = "starred_papers.csv"
# starred_papers.to_csv(output_file, index=False)

In [ ]:
thesis_papers = available_papers[available_papers["Is the paper a thesis (Bachelor's, Master's, or PhD)?"] == "Yes"]

# Also, check for any occurrence of the word 'thesis' in the "Optional short description"
thesis_keyword_papers = available_papers[available_papers["Optional short description"].str.contains("thesis", case=False, na=False)]
all_thesis_papers = pd.concat([thesis_papers, thesis_keyword_papers]).drop_duplicates()


thesis_percentage = len(all_thesis_papers) / len(available_papers) * 100
print("Number of paper by core-authors: ",len(all_thesis_papers[all_thesis_papers["Does the paper have at least 1 current OpenML Core Member as co-author?"] == "Yes"]))

print(len(all_thesis_papers), thesis_percentage)


In [110]:
# Differentiate in skill level between core members and non-core members
